In [1]:
# Importamos librerias necesarias
import numpy as np

import torch

from tqdm import tqdm
from classical_shadow import random_state
from tomography import pauli_basis, gellmann_basis, simulate_expectation, standard_tomography, fidelity

In [2]:
N_exp_test  = [2000, 4000, 6000]
p_basis     = pauli_basis(N = 1)
gm_basis    = gellmann_basis(d = 2)

f1 = []
f2 = []

for n_exp in tqdm(N_exp_test, desc = "Generating data for comparisson"):
    # Prepare a list to save the results
    fidelities = [], []

    # We repeat the same process multiple times using different randomly generated quantum states
    for _ in range(100):
        # Choose a random quantum state
        psi_true = random_state()

        # -------------------------------
        # Then, we do two tomographies more using standard tomograhy, one using Pauli basis and the other using Gellmann matrices
        # -------------------------------
        # Prepare the proper density matrices related with the psi_true vector
        rho_true = psi_true.detach().numpy()
        rho_true = np.kron(rho_true.conj().T, rho_true).reshape(2, 2)

        # Simulate the different expectations.
        pauli_expectation   = np.array([simulate_expectation(rho = rho_true, O = pauli_obs, n_exp = int(n_exp / 3)) for pauli_obs in p_basis])
        gm_expectation      = np.array([simulate_expectation(rho = rho_true, O = gm_obs, n_exp = int(n_exp / 3)) for gm_obs in gm_basis])

        # Then we do the standard tomography
        rho_standard_pauli = standard_tomography(set_measure_basis = p_basis, expectation_values = pauli_expectation)
        rho_standard_gm = standard_tomography(set_measure_basis = gm_basis, expectation_values = gm_expectation)

        # Compute the fidelities
        fidelity_p  = fidelity(rho = rho_true, sigma = rho_standard_pauli)
        fidelity_gm = fidelity(rho = rho_true, sigma = rho_standard_gm)

        # Save the results in the list

        # For Standard tomography
        fidelities[0].append(fidelity_p)
        fidelities[1].append(fidelity_gm)

    f1.append((np.mean(fidelities[0]), np.std(fidelities[0])))
    f2.append((np.mean(fidelities[1]), np.std(fidelities[1])))

Generating data for comparisson: 100%|██████████| 3/3 [00:15<00:00,  5.21s/it]


In [3]:
f1

[(0.9950878776917774, 0.006462905470503137),
 (0.9959122309387783, 0.005610353091395169),
 (0.997410992193708, 0.004106613343911327)]

In [4]:
f2

[(0.9952160937315334, 0.008186219719577042),
 (0.9976192186048151, 0.003910819832722953),
 (0.9974811978575154, 0.0037998130211662146)]